In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip -q install --upgrade langchain openai
!pip -q install --upgrade langchain_community
!pip -q install --upgrade llamaapi
!pip -q install --upgrade ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 24.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:0000:0100:01


In [11]:
!pip -q install -U langchain-ollama

0

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
import os
get_ipython().system = os.system

In [5]:
!ollama serve &

0

In [7]:
# !ollama pull deepseek-r1

0

In [6]:
!ollama pull mistral

0

In [7]:
import json
from tqdm import tqdm
import openai
from openai import OpenAI
from llamaapi import LlamaAPI
import ollama

# from langchain_community.chat_models import ChatOllama
# from langchain.document_loaders import WikipediaLoader
# # from langchain_community.llms import Ollama
# from langchain_ollama import OllamaLLM
# from langchain.chains import LLMChain
# from langchain.prompts.chat import (ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate)
# from langchain import PromptTemplate

In [14]:
# llm = OllamaLLM(model="deepseek-r1", temprature=1.3) 

In [8]:
# Path to your input and output files
input_file = "/kaggle/input/task-2-ea-mt-entity-aware-machine-translation/semeval.test_hidden/test_without_targets/ar_AE.jsonl"
output_file = "translated_data_en_ar_ollama-mistral-v2.jsonl"

In [9]:
def translate_text(obj):
    prompt = f"""
    Translate the following English text into Arabic with a more attention on accurately translating the given name entities. 
    Do not add extra text.
    
    Entity Types: {', '.join(obj['entity_types'])}
    Text: "{obj['source']}"
    
    Output the result in JSON format:
    {{
        "id": "{obj['id']}",
        "predicted_text": "<translated_text>"
    }}
    """
    
    response = ollama.chat(
        model="mistral",
        messages=[{"role": "system", "content": "You are a translation assistant."},
                  {"role": "user", "content": prompt}],
        stream=False
    )
    
    # Extract generated text and parse JSON
    try:
        # print(response)
        output_json = json.loads(response.message.content)
        return output_json
    except json.JSONDecodeError:
        print(f"Error parsing response for ID: {obj['id']}")
        return None

In [10]:
with open(input_file, "r", encoding="utf-8") as f:
    total_lines = sum(1 for _ in f)

In [11]:
# Read input file and translate each object
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    with tqdm(total=total_lines, desc="Generating Predictions", colour="blue") as pbar:
            for line in infile:
                obj = json.loads(line.strip())
                translated_obj = translate_text(obj)
                if translated_obj:
                    outfile.write(json.dumps(translated_obj, ensure_ascii=False) + "\n")
                pbar.update(1)

print(f"Translation completed! Output saved to {output_file}")

Generating Predictions:   1%|          | 28/4547 [00:54<3:53:15,  3.10s/it]

Error parsing response for ID: 1a40ad2d54db3964


Generating Predictions:   1%|          | 29/4547 [00:58<4:24:15,  3.51s/it]

Error parsing response for ID: 6c25128cc21690da


Generating Predictions:   1%|          | 36/4547 [01:13<2:51:48,  2.29s/it]

...

Error parsing response for ID: 667253fd123f2743


Generating Predictions: 100%|█████████▉| 4532/4547 [2:31:54<00:36,  2.44s/it]

Error parsing response for ID: 6d604c38a180c4b3


Generating Predictions: 100%|██████████| 4547/4547 [2:32:24<00:00,  2.01s/it]

Translation completed! Output saved to translated_data_en_ar_ollama-mistral-v2.jsonl
